In [15]:
import tensorflow as tf
import os
import numpy as np

tf.reset_default_graph()
tf.set_random_seed(777)

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)
#load data
xy = np.genfromtxt('/Users/yeseo/Desktop/taxi_data/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)

#data preprocessing
train_size = int(len(xy)*0.7)

seq_length =6
data_dim =27
hidden_dim = 27
output_dim = 25
learning_rate = 0.1
iterations = 1001

xy= xy[:,:27]
test_set = xy[train_size:]
a = xy[:,:2]
b = xy[:,2:]
data_max = np.max(b,1)
data_min = np.min(b,1)
b= MinMaxScaler(b)
xy = np.hstack((a,b))
train_set = xy[:train_size]

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)


saver = tf.train.import_meta_graph("/Users/yeseo/ML_with_Taxidata/code/save_model_code/model/my_model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess,"/Users/yeseo/ML_with_Taxidata/code/save_model_code/model/my_model.ckpt")
    
    graph = tf.get_default_graph()
    X = graph.get_tensor_by_name("X:0")
    Y = graph.get_tensor_by_name("Y:0")

    Y_pred = graph.get_tensor_by_name("rnn/fully_connected/BiasAdd:0")
    
    test_predict = sess.run(Y_pred, feed_dict = {X:validationX})

INFO:tensorflow:Restoring parameters from /Users/yeseo/ML_with_Taxidata/code/save_model_code/model/my_model.ckpt


In [16]:
test_predict

array([[0.519983  , 0.5357133 , 0.7311876 , ..., 0.62834936, 0.8134205 ,
        0.77985823],
       [0.48948288, 0.5064425 , 0.7059424 , ..., 0.59972465, 0.8103568 ,
        0.7986638 ],
       [0.46200874, 0.4708667 , 0.6635073 , ..., 0.5821145 , 0.80588466,
        0.7984813 ],
       ...,
       [0.65016776, 0.6609901 , 0.62372655, ..., 0.61395335, 0.3835152 ,
        0.27562097],
       [0.6763084 , 0.6765678 , 0.64603007, ..., 0.64307237, 0.43519932,
        0.31744656],
       [0.69684863, 0.6880646 , 0.6709062 , ..., 0.67722094, 0.46239683,
        0.3378895 ]], dtype=float32)